## Definição da Classe Software

In [1]:
import numpy as np
from PIL import Image
import time

#lists = ["1.jpg","2.jpg","3.jpg","4.jpg","5.jpg"]
class software:

    def __init__(self, resolucao):
        self.tempoTotal = []
        self.resolucao = resolucao
        self.MSE = []
        self.tempoDoCalculo = []
        self.tempoSetarImagem = []

    def getResolucao(self):
        return str(self.resolucao)

    def setTempoSetarImagem(self, t1,t2):
        self.tempoSetarImagem.append(t2-t1)

    def getTempoSetarImagem(self):
        return np.mean(self.tempoSetarImagem)
    
    def getTempoSetarImagemIndice(self,indice):
        return self.tempoSetarImagem[indice]

    def setTempoTotal(self, t1, t2):
        self.tempoTotal.append(t2 - t1)
        
    def getTempoTotalMedia(self):
        return np.mean(self.tempoTotal)
    
    def getTempoTotalMediaIndice(self,indice):
        return self.tempoTotal[indice]

    def setMSE(self, original,other):
        difference = np.subtract(original, other)
        MSE = np.square(difference).mean()
        self.MSE.append(MSE)

    def getMediaMSE(self):
        return np.mean(self.MSE)
    
    def getMediaMSEIndice(self,indice):
        return self.MSE[indice]

    def setTempoDeCalcular(self, t1, t2):
        self.tempoDoCalculo.append(t2-t1)

    def getMediaTempoDeCalcular(self):
        return np.mean(self.tempoDoCalculo)
    
    def getMediaTempoDeCalcularIndice(self,indice):
        return self.tempoDoCalculo[indice]
    
    def getDesvioPadraoTempoTotal(self):
      return np.std(self.tempoTotal)

    def GrayScaleSW(self, img, width, height):
        #img = file.load()
        #width, height = file.size

        newImageSoftware = Image.new('RGB', (width, height))
        setPixelSoftware = newImageSoftware.load()

        for x in range(width):
            for y in range(height):
                R_converted = img[x,y][0] * 0.299
                G_converted = img[x,y][1] * 0.587
                B_converted = img[x,y][2] * 0.114
                Y = int(R_converted + G_converted + B_converted)
                setPixelSoftware[x, y] = (Y, Y, Y)
        return newImageSoftware

    
    def GrayScaleSW_Vec(self, input_array, width, height):
        constants = np.array([0.299, 0.587, 0.114])
        newImageSoftware = Image.new('RGB', (width, height))
        
        RGB_converted = input_array * constants
        MatY = RGB_converted.sum(axis=2, dtype=np.uint8)        
        GraySW = np.repeat(MatY[:, :, np.newaxis], 3, axis=2)
        
        newImageSoftware = Image.fromarray(GraySW)
        return newImageSoftware

## Teste SW - Silvio

In [2]:
from PIL import Image

#imageInSoftware = software("640x360") #setar resolução: 1920x1080 1280x720 640x360
#imageInSoftware = software("1280x720") #setar resolução: 1920x1080 1280x720 640x360
imageInSoftware = software("1920x1080") #setar resolução: 1920x1080 1280x720 640x360
for i in range(1):
        
    start_time = time.time()

    file = Image.open("../images/imagem-"+imageInSoftware.getResolucao()+".jpg")
    img = file.load()
    width, height = file.size
    input_array = np.array(file)
    
    #### Sequential software
    result = imageInSoftware.GrayScaleSW(img, width, height)
    result.save("../images/imagem-"+str(imageInSoftware.getResolucao())+"-SW.jpg")
    
    #### Vectorized software
    #result = imageInSoftware.GrayScaleSW_Vec(input_array, width, height)
    #result.save("../images/imagem-"+str(imageInSoftware.getResolucao())+"-SW-Vec.jpg")
    
    stop_time = time.time()

    imageInSoftware.setTempoTotal(start_time,stop_time)
        
print(imageInSoftware.getTempoTotalMedia())
print(imageInSoftware.getDesvioPadraoTempoTotal())

17.695154666900635
0.0


## Definição da Classe Hardware

In [48]:
import numpy as np
import pynq.lib.dma

from pynq import Overlay
from pynq import Xlnk


class hardware:

    def __init__(self, resolucao, bitstream_name):
        self.resolucao = resolucao
        
        res = self.resolucao.split('x')
        self.width = int(res[0])
        self.height = int(res[1])
        
        self.overlay = Overlay(bitstream_name)
        self.dma = self.overlay.axi_dma_0
        self.registers = self.overlay.GrayScale_Accel_pipe_0
        
        self.xlnk = Xlnk()
        self.in_buffer = self.xlnk.cma_array(shape=(self.height, self.width, 3), dtype=np.uint8, cacheable=1)
        self.out_buffer = self.xlnk.cma_array(shape=(self.height, self.width, 3), dtype=np.uint8, cacheable=1)
                
        self.tempoTotal = []
        self.tempoBufferDMA = []
        self.MSE = []
        self.tempoDoCalculo = []
        self.tempoSetarImagem = []
        self.limparBuffer = []
        self.lerImagem = []
        self.salvarImagem = []
    
    def setTempoSalvarImagem(self, t1, t2):
        self.salvarImagem.append(t2-t1)
    
    def getTempoSalvarImagem(self):
        return np.mean(self.salvarImagem)
    
    def getTempoSalvarImagemIndice(self,indice):
        return self.salvarImagem[indice]
    
    def setTempoLerImagem(self, t1, t2):
        self.lerImagem.append(t2-t1)
    
    def getTempoLerImagem(self):
        return np.mean(self.lerImagem)
    
    def getTempoLerImagemIndice(self,indice):
        return self.lerImagem[indice]
        
    def setTempoLimparBuffer(self, t1,t2):
        self.limparBuffer.append(t2-t1)

    def getTempoLimparBuffer(self):
        return np.mean(self.limparBuffer)
    
    def getTempoLimparBufferIndice(self,indice):
        return self.limparBuffer[indice]
    
    def getResolucao(self):
        return str(self.resolucao)

    def setTempoSetarImagem(self, t1,t2):
        self.tempoSetarImagem.append(t2-t1)

    def getTempoSetarImagem(self):
        return np.mean(self.tempoSetarImagem)
    
    def getTempoSetarImagemIndice(self,indice):
        return self.tempoSetarImagem[indice]

    def setTempoTotal(self, t1, t2):
        self.tempoTotal.append(t2 - t1)
        
    def getTempoTotalMedia(self):
        return np.mean(self.tempoTotal)
    
    def getTempoTotalMediaIndice(self,indice):
        return self.tempoTotal[indice]

    def setTempoBufferDMA(self,t1,t2):
        self.tempoBufferDMA.append(t2-t1)

    def getMediaTempoBufferDMA(self):
        return np.mean(self.tempoBufferDMA)
    
    def getMediaTempoBufferDMAIndice(self,indice):
        return self.tempoBufferDMA[indice]

    def setMSE(self, original,other):
        difference = np.subtract(original, other)
        MSE = np.square(difference).mean()
        self.MSE.append(MSE)

    def getMediaMSE(self):
        return np.mean(self.MSE)
    
    def getMediaMSEIndice(self,indice):
        return self.MSE[indice]

    def setTempoDeCalcular(self, t1, t2):
        self.tempoDoCalculo.append(t2-t1)

    def getMediaTempoDeCalcular(self):
        return np.mean(self.tempoDoCalculo)
    
    def getMediaTempoDeCalcularIndice(self,indice):
        return self.tempoDoCalculo[indice]
    
    def getDesvioPadraoTempoTotal(self):
      return np.std(self.tempoTotal)
        
    def GrayScaleHW(self, input_array):#, width, height):
        #xlnk = Xlnk()
        #in_buffer = xlnk.cma_array(shape=(height, width, 3), dtype=np.uint8, cacheable=1)
        #out_buffer = xlnk.cma_array(shape=(height, width, 3), dtype=np.uint8, cacheable=1)
        self.in_buffer[0:self.width*self.height*3] = input_array
        
        ############
        self.dma.sendchannel.transfer(self.in_buffer)
        self.dma.recvchannel.transfer(self.out_buffer)
        self.registers.write(0x00,1) #coloca o valor 1 no registrador 0 para start
        self.dma.sendchannel.wait()
        self.dma.recvchannel.wait()
        ###########

        newImage = Image.new('RGB', (self.width, self.height))
        #setPixelSoftware = newImageSoftware.load()
        
        newImage = Image.fromarray(self.out_buffer)
        return newImage


## Teste Hardware

In [49]:
import time
from PIL import Image

#setar resolução: 1920x1080 1280x720 640x360
#imageInHardware = hardware('640x360', '../ip_repo/design_pipe.bit') 

imageInHardware = hardware('640x360', '../ip_repo/design_pipe_2.bit')
#imageInHardware = hardware('1280x720', '../ip_repo/design_pipe_2.bit')
#imageInHardware = hardware('1920x1080', '../ip_repo/design_pipe_2.bit')

#imageInHardware = hardware('640x360', '../ip_repo/design_1.bit') 
#imageInHardware = hardware("1280x720", '../ip_repo/design_1.bit')
#imageInHardware = hardware("1920x1080", '../ip_repo/design_1.bit')
for i in range(1):
        
    file = Image.open("../images/imagem-"+imageInHardware.getResolucao()+".jpg")
    width, height = file.size
    input_array = np.array(file)

    start_time = time.time()
    #out_buffer = imageInHardware.GrayScaleHW(input_array)
    result = imageInHardware.GrayScaleHW(input_array)#, width, height)
    #result = Image.fromarray(out_buffer)
    stop_time = time.time()

    result.save("../images/imagem-"+str(imageInHardware.getResolucao())+"-HW.jpg")
    

    imageInHardware.setTempoTotal(start_time,stop_time)
        
print(imageInHardware.getTempoTotalMedia())
print(imageInHardware.getDesvioPadraoTempoTotal())
#result

0.005857944488525391
0.0


## Teste comparativo apenas do cálculo

In [51]:
import time
from PIL import Image

def MSE(original, other):
    difference = np.subtract(original, other)
    MSE = np.square(difference).mean()
    return MSE

#setar resolução:  640x360  1280x720   1920x1080
resolucao = "640x360"
#resolucao = "1280x720"
#resolucao = "1920x1080"

#imageInHardware = hardware(resolucao, '../ip_repo/design_1.bit') 
#imageInHardware = hardware(resolucao, '../ip_repo/design_pipe.bit') 
imageInHardware = hardware(resolucao, '../ip_repo/design_pipe_2.bit')
imageInSoftware = software(resolucao)

file = Image.open("../images/imagem-"+resolucao+".jpg")
img = file.load()
width, height = file.size
input_array = np.array(file)

### Hardware
start_hw = time.time()
result_hw = imageInHardware.GrayScaleHW(input_array)#, width, height)
stop_hw = time.time()
tempo_hw = stop_hw - start_hw

### Sequential software
start_sw = time.time()
result_sw = imageInSoftware.GrayScaleSW(img, width, height)
stop_sw = time.time()
tempo_sw = stop_sw - start_sw

### Vectorizated software
start_sw_vec = time.time()
result_sw_v = imageInSoftware.GrayScaleSW_Vec(input_array, width, height)
stop_sw_vec = time.time()
tempo_sw_vec = stop_sw_vec - start_sw_vec
       
print(f"Tempo em segundos para resolução: {resolucao}")
print(f"Software: {tempo_sw}")
print(f"Software Vec: {tempo_sw_vec}")
print(f"Hardware: {tempo_hw}")

print("##### Comparativo da qualidade das imagens")
print(f"Hardware x Software: {MSE(result_hw, result_sw)}")
print(f"Hardware x Software Vec: {MSE(result_hw, result_sw_v)}")
print(f"Software x Software Vec: {MSE(result_sw, result_sw_v)}")


Tempo em segundos para resolução: 640x360
Software: 1.8261661529541016
Software Vec: 0.2742166519165039
Hardware: 0.006288051605224609
##### Comparativo da qualidade das imagens
Hardware x Software: 51.86186197916667
Hardware x Software Vec: 52.129292534722225
Software x Software Vec: 1.3307986111111112


## Teste de médias e desvio padrão

In [71]:
import time
from PIL import Image

resolucao = "640x360"
#resolucao = "1280x720"
#resolucao = "1920x1080"

imageInHardware = hardware(resolucao, '../ip_repo/design_1.bit') 
imageInSoftware = software(resolucao)

file = Image.open("../images/imagem-"+resolucao+".jpg")
img = file.load()
width, height = file.size
input_array = np.array(file)

for i in range(10):
    start_time = time.time()
    #result = imageInHardware.GrayScaleHW(input_array, width, height)
    #result = imageInSoftware.GrayScaleSW_Vec(input_array, width, height)
    result = imageInSoftware.GrayScaleSW(img, width, height)
    stop_time = time.time()

    #imageInHardware.setTempoTotal(start_time,stop_time)
    imageInSoftware.setTempoTotal(start_time,stop_time)
        
#result.save("../images/imagem-"+str(imageInHardware.getResolucao())+"-HW.jpg")
#print(imageInHardware.getTempoTotalMedia())
#print(imageInHardware.getDesvioPadraoTempoTotal())

#result.save("../images/imagem-"+str(imageInSoftware.getResolucao())+"-SW.jpg")
result.save("../images/imagem-"+str(imageInSoftware.getResolucao())+"-SW-Vec.jpg")
print(imageInSoftware.getTempoTotalMedia())
print(imageInSoftware.getDesvioPadraoTempoTotal())



1.8356171607971192
0.00248474946428569
